In [237]:
import pandas as pd

data = pd.read_excel('data_set.xlsx')

def replace_binary_value(b):
    if b and pd.notnull(b):
        return 1
    else:
        return -1


data.iloc[:, 12:16] = data.iloc[:, 12:16].agg(lambda row: row.agg(replace_binary_value, axis=0), axis=1)
data_tul_obl = data[data['REGION'] == 'Тульская область']

data_tula = data_tul_obl[data_tul_obl['CITY'] == 'ТУЛА']

X = data_tula[data_tula['Postamat_daily'].notnull()].iloc[:, 5:18]

del X['cashbox_trend']
del X['POPULATION']
del X['macro_salary_avg_yearly']


y = data_tula[data_tula['Postamat_daily'].notnull()].Postamat_daily
y = (y - y.mean()) / y.std()

X.fillna(0, inplace=True)
X = (X - X.mean()) / X.std()




In [238]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

lasso = Lasso(0.005)

X_train, X_test, y_train, y_test = train_test_split(X, y)
X.info()

lasso.fit(X, y)
lasso.coef_

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 8 to 249
Data columns (total 9 columns):
Postamat_trend           26 non-null float64
cashbox_MAX              26 non-null float64
hh_500                   26 non-null float64
value1                   26 non-null float64
value2                   26 non-null float64
NEAR_Malls               26 non-null float64
NEAR_Business_centers    26 non-null float64
NEAR_metro_rjd           26 non-null float64
NEAR_Stations            26 non-null float64
dtypes: float64(9)
memory usage: 2.0 KB


array([-0.        , -0.50334116,  0.23572687,  0.        , -0.05642533,
        1.08186438, -0.24518251, -0.43567001,  0.05688665])

In [239]:
X.drop(labels=['Postamat_trend', 'value1'], axis=1, inplace=True)
lasso.fit(X, y)
lasso.coef_

array([-0.50333272,  0.23573013, -0.0564297 ,  1.08186865, -0.24519041,
       -0.43567209,  0.05688678])

In [240]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
from sklearn.tree import ExtraTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor


def test_regressor(X, y, regressor):
    mean_mse = 0
    mean_mae = 0
    for i in range(100):
        X_train , X_test, y_train, y_test = train_test_split(X, y)
        regressor.fit(X_train, y_train)
        mean_mse += mse(y_test, regressor.predict(X_test))
        mean_mae += mae(y_test, regressor.predict(X_test))
    print('MSE = ', mean_mse/100)
    print('MAE = ', mean_mae/100)


def getBenchmark(X, y, lasso_coef=0.05):
    lasso = Lasso(lasso_coef)
    ridge = Ridge()
    lin_reg = LinearRegression()
    des_tree = DecisionTreeRegressor()
    extra_tree = ExtraTreeRegressor()
    knn = KNeighborsRegressor()
    
    
    print('LinReg:')
    test_regressor(X, y, lin_reg)
    print('Lasso:')
    test_regressor(X, y, lasso)
    print('Ridge:')
    test_regressor(X, y, ridge)
    print('ExtraTree:')
    test_regressor(X, y, extra_tree)
    print('KNN:')
    test_regressor(X, y, knn)
    print('DesTree:')
    test_regressor(X, y, des_tree)

In [241]:
getBenchmark(X, y, 0.005)

LinReg:
MSE =  1.0378043167662816
MAE =  0.8052508734569842
Lasso:
MSE =  1.2078763885986055
MAE =  0.8726693359515226
Ridge:
MSE =  1.007520271743893
MAE =  0.7963248477498145
ExtraTree:
MSE =  1.69709296019091
MAE =  1.0527887586165967
KNN:
MSE =  1.0524937628810072
MAE =  0.7858280247824547
DesTree:
MSE =  1.7986224102397224
MAE =  1.0628248764299106


In [253]:
def getPredict(data, regressor, labels_to_delete, target_name):
    X = data[data[target_name].notnull()].iloc[:, 5:18]
    X.drop(labels=labels_to_delete, axis=1, inplace=True)
    y = data[data[target_name].notnull()][target_name]
    regressor.fit(X, y)

    df = data[data[target_name].isnull()].iloc[:, 5:18]
    df.drop(labels=labels_to_delete, axis=1, inplace=True)
    arr = regressor.predict(df)
    
    forResult = data[data[target_name].isnull()]['ADDRESS']
    indexes = [x for x in range(len(arr))]
    forResult.index = indexes
    result = {}
    for i in range(len(arr)):
        result[forResult.loc[i]] = arr[i]

    return result

In [254]:
l = ['Postamat_trend', 'value1', 'cashbox_trend',\
     'POPULATION', 'macro_salary_avg_yearly']
data_tula['cashbox_MAX'] = 0
target = 'Postamat_daily'
getPredict(data_tula, Ridge(), l, target)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


{'г.Тула, Кутузова ул., 13': 5.470818759440553,
 'г.Тула, Степанова ул., 89 А': 5.600541813540753,
 'г.Тула, Пузакова ул., 1': 3.2359171474978043,
 'г.Тула, Болдина ул., 98': 3.3058142999072406,
 'г.Тула, Макаренко ул., 9 Б': 5.512491738015392,
 'г.Тула, Калужское ш., 2': 5.111588950416127,
 'г.Тула, Октябрьская ул, 48 Б': 5.19503501379303,
 'г.Тула, Советская ул, 72': 4.5334724851558725,
 'г.Тула, Болдина ул': 5.076856653422588,
 'г.Тула, Металлургов ул, 43': 2.8743715260615272,
 'г.Тула, Металлургов 2-й пр-д, 3А': 2.854394388769326,
 'г.Тула, Кутузова ул, 102': 2.6735798226943097,
 'г.Тула, Демидовская ул., 68': 5.2812215792372115,
 'г.Тула, Большая ул, 8А': 2.078232807908489,
 'г.Тула, Городской пер., 54': 2.226751552114322,
 'г.Тула, Калужское ш., 1': 4.978394460086321,
 'г.Тула, Пушкина ул., 9': 2.405420360851686,
 'г.Тула, Одоевское ш, 17А': 2.1722676004055725,
 'г.Тула, Скуратовский мкр, 7': 2.116435272919181,
 'г.Тула, Московская ул, 34': 0.2669345349635783,
 'г.Тула, Серебровс